# ご参考：ニューラルネットワークの影響変数

影響変数が容易に確認できるよう、訓練データの量を制限しています。

また、影響変数にかんする補足記述を追加しております。


#### 影響変数（coefs\_）についての説明

http://scikit-learn.org/dev/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier

~~~
Attributes:
    coefs_ : list, length n_layers - 1
        The ith element in the list represents the weight matrix corresponding to layer i.
~~~

各レイヤー（中間レイヤーまたは出力レイヤー）における重み付けマトリックスとのことです。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

### (1-1) テストデータをコピー

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_daikin_conversation.csv',
    'test_benefitone_conversation.csv',
    'test_septeni_conversation.csv',
    'test_ptna_conversation.csv',
]
temp_path = TestTool.copy_testdata_csv(learning_dir, csv_file_names)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]


In [3]:
b, _ = os.path.splitext(temp_path[2])
copied_csv_file_paths = b + '.limited.csv'
copied_csv_file_paths

'/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.limited.csv'

In [4]:
import subprocess

cmd = "head -28 " + temp_path[0] + " > " + copied_csv_file_paths
subprocess.call(cmd, shell=True)

0

## (2) モデルの準備

学習(GridSearchCV.fit)--->評価(Evaluator.evaluate)の流れで実行させます。

グリッドサーチは省略しております。

In [5]:
from time import time
import numpy as np

from sklearn.grid_search import GridSearchCV
from sklearn.neural_network import MLPClassifier

from learning.core.evaluator import Evaluator

def fit_and_cross_validation(path):
    '''
        訓練データのTF-IDFベクターを作成
    '''
    basename = os.path.basename(path)
    print("prepare_tf_idf_vectors: dataset=%s..." % basename)
    t0 = time()

    X, y, vectorizer = TestTool.prepare_tf_idf_vectors(path)
    print("prepare_tf_idf_vectors: done in %0.3fs." % (time() - t0))

    '''
        訓練データ全体を使用して学習実施
        レイヤーはデフォルトの1層
        レイヤーに100件ユニットを生成する設定
    '''
    print("MLPClassifier: fitting...")
    t0 = time()

    cls = MLPClassifier(hidden_layer_sizes=(100,), max_iter=10000,
                        activation='logistic', shuffle=False, random_state=0)
    estimator = cls.fit(X, y)
    print("MLPClassifier: done in %0.3fs." % (time() - t0))

    ''' 
        クロスバリデーション（モデル評価フェーズ）を実施
        プロダクションと同様、Evaluator クラスを使用して評価します
    '''
    print("Evaluator: evaluating...")
    t0 = time()

    evaluator = Evaluator()
    evaluator.evaluate(estimator, X, y, threshold=0.5)
    print("Evaluator: done in %0.3fs." % (time() - t0))
    
    return (basename, X, y, vectorizer, estimator, evaluator)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### (2-1) モデル作成／評価

In [6]:
classifier = fit_and_cross_validation(copied_csv_file_paths)

2017/04/07 PM 03:35:29 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 27
2017/04/07 PM 03:35:29 TextArray#__init__ start
2017/04/07 PM 03:35:29 TextArray#to_vec start
2017/04/07 PM 03:35:29 TextArray#to_vec end


prepare_tf_idf_vectors: dataset=test_septeni_conversation.limited.csv...
prepare_tf_idf_vectors: done in 0.034s.
MLPClassifier: fitting...


2017/04/07 PM 03:35:30 self.threshold: 0.5


MLPClassifier: done in 0.861s.
Evaluator: evaluating...


2017/04/07 PM 03:35:30 Evaluator#evaluate#elapsed time: 785.332918 ms
2017/04/07 PM 03:35:30 accuracy: 1.0


1.0
Evaluator: done in 0.788s.


### (2-2) 結果の確認

In [7]:
basename, X, y, vectorizer, estimator, evaluator = classifier
params = estimator.get_params()

print('[%s] best parameter: hidden_layer_sizes=%s, accuracy=%0.6f' % (
    basename, str(params['hidden_layer_sizes']), evaluator.accuracy
))

[test_septeni_conversation.limited.csv] best parameter: hidden_layer_sizes=(100,), accuracy=1.000000


## (3) 影響変数の確認

ここからが本題になります

今回使用した MLP は、３層のレイヤーから構成されているモデルです。


- 入力レイヤー：147件のユニット（featureの件数と同じ）


- 中間レイヤー：100件のユニット（MLP の hidden_layer_sizes=(100,) で指定した件数と同じ）


- 出力レイヤー：28件のユニット（ラベルの件数と同じ）


これらのユニット同士が、レイヤーとレイヤーの間でどの程度強く結びついているか（重み付けが与えられているか）を確認していきたいと思います。

### (3-1) 予測結果の抽出

In [8]:
y_pred = estimator.predict(X)
probabilities = estimator.predict_proba(X)
max_probabilities = np.max(probabilities, axis=1)

### (3-2) データのfeatureを確認

In [9]:
for i, _ in enumerate(y):
    arr = X[i].toarray()[0]
    dump_str = TestTool.dump_features(arr, vectorizer.vocabulary_)
    print('index=%d [label=%d] %s' % (
        i, y[i], dump_str))

index=0 [label=3397] [2010=0.258 bcc=0.258 outlook=0.258 する=0.516 メール=0.516 宛=0.258 自分=0.258 送信=0.258 運用=0.258]
index=1 [label=3398] [2010=0.408 outlook=0.408 メール=0.408 作り方=0.408 教える=0.408 署名=0.408]
index=2 [label=3399] [2010=0.378 outlook=0.378 thunderbird=0.378 テンプレート=0.378 メール=0.378 同等=0.378 機能=0.378]
index=3 [label=3400] [2010=0.224 outlook=0.224 thunderbird=0.224 できる=0.224 ない=0.224 アドレス=0.447 候補=0.224 入れる=0.224 出る=0.224 場合=0.224 姓=0.224 宛先=0.224 帳=0.224 検索=0.224 機能=0.224 簡単=0.224 者=0.224]
index=4 [label=3401] [outlook=0.204 いちいち=0.204 こちら=0.204 する=0.408 できる=0.204 ない=0.204 わずらわしい=0.204 アドレス=0.408 デフォルト=0.204 メールアドレス=0.204 一覧=0.408 意図=0.204 検索=0.204 設定=0.204 選択=0.204]
index=5 [label=3402] [2010=0.447 outlook=0.447 メール=0.447 可能=0.447 編集=0.447]
index=6 [label=3940] [2010=0.408 outlook=0.408 メール=0.408 大変=0.408 探す=0.408 未読=0.408]
index=7 [label=3404] [2010=0.378 outlook=0.378 thunderbird=0.378 トレイ=0.378 済=0.378 送信=0.378 違い=0.378]
index=8 [label=3405] [2010=0.354 outlook=0.354 する=0.354 フ

### (3-3) 調査用サンプルの選定

#### この中から、ラベル 3402 をサンプルとして、影響変数を追跡してみます。

CSVにおける内容は下記の通り
~~~
1385823
Outlook2010にて、メールの再編集は可能ですか？
3402
"方法については『【Outlook2010操作マニュアル 基本編】 送信済メールの再利用（メールの再送）』を参照してください。 

http://www.intra.daikin.co.jp/office365/ol2010/Basic.html?cid=B007

【手順】
１．再利用したいメールをダブルクリックします。 
２．『メッセージ』リボンから『アクション』の▼を押下し、『このメッセージを再送』を押下します。 
３．メール編集画面が表示されますので、以降の操作は『メールの送信』を参照しメールを送信してください。 
※古いメールを再利用（再送）すると、送信エラーが発生することがありますので、
　その場合は、再利用（再送）ではなく新規作成をして下さい。"
~~~

#### また、ラベル 3402 に類似した質問（3940）を、比較用のサンプルとします。

CSVにおける内容は下記の通り
~~~
1385824	
Outlook2010にて、未読メールを探すのが大変です。	
3940	
"方法については『【Outlook2010操作マニュアル 応用編】 検索フォルダ(未読のメールのフォルダを作成する)』を参照してください。 
http://www.intra.daikin.co.jp/office365/ol2010/Applied.html?cid=C007

【手順】
１．検索フォルダを右クリックし、『新しい検索フォルダ』を押下します。
２．『未読のメール』を選択します。 
３．『OK』ボタンを押下します。
４．『未読のメール』検索フォルダーが作成されていることを確認します。

※検索フォルダでは未読のメール以外にも、フラグの設定されたメールなど様々な条件のメールを検索条件とすることができます。
※検索フォルダに表示されているメールは検索結果となりますので、メールは元のフォルダから移動はしておりません。"
~~~

#### ボキャブラリIDを取得しておきます

In [10]:
[(k,v) for (k,v) in vectorizer.vocabulary_.items() if k in [
    '2010','outlook','メール','編集','可能','未読','探す','大変'
]]

[('未読', 105),
 ('編集', 123),
 ('大変', 85),
 ('可能', 78),
 ('メール', 43),
 ('探す', 100),
 ('outlook', 10),
 ('2010', 1)]

### (3-3) MLP内における feature〜ユニット間の重み付けを確認

MLPにおいては、coefs\_ という変数に、学習結果（最適化された影響変数）が格納されています。

coefs\_ の内部レイヤーに対応する要素（estimator.coefs\_[0]）は、feature数 × ユニット数（100件）の２次元配列になっています。

In [11]:
np.size(estimator.coefs_[0], axis=0) # <--- featureの数になります

147

In [12]:
np.size(estimator.coefs_[0], axis=1) # <--- ユニットの数になります

100

こちらの estimator.coefs\_[0] を参照すると、あるfeatureが、各々のユニットに対して、どれだけ影響の強さを持っているか（＝重み付けがされているか）が判断できるかと存じます。

逆の見方をすると・・・各々のfeatureが、どのユニットへ誘導されやすいか、ということを判断する手助けになるかと考えております。

<B>この影響変数の設定じたいは、MLP の fit 関数実行中に（最適化処理により）行なわれるものなので、パラメータによる個別調整は不可能です。</b>

In [13]:
'''
     feature「outlook」に対する、ユニットごとの影響変数
'''
outlook = estimator.coefs_[0][10] # ('outlook', 10)
outlook

array([-0.24349488, -0.58650797, -0.62772164, -0.18579832, -0.68337962,
        0.45881705,  0.48999331,  0.33866849,  0.46231785,  0.67383954,
       -0.38977021, -0.61523463, -0.60205338, -0.49270935,  0.34617328,
        0.36257249,  0.16874911,  0.04544902, -0.59871436,  0.68734688,
        0.45151423, -0.52513091, -0.43037663, -0.37098965, -0.43685076,
       -0.27314701,  0.55091456, -0.7087831 , -0.4324771 , -0.5869078 ,
       -0.56536217,  0.3496927 , -0.39246992,  0.52282408,  0.44584184,
       -0.53519944,  0.27778031, -0.61097927, -0.17362748,  0.62312865,
       -0.00094078, -0.58308338,  0.37934531,  0.6290719 , -0.70525309,
       -0.49715026, -0.49586602,  0.67151102,  0.51786721,  0.61357513,
        0.35009992,  0.40764122, -0.19257504, -0.61780065, -0.41293255,
       -0.59688374,  0.6592304 ,  0.61203625, -0.35644356, -0.4130698 ,
       -0.43783877, -0.51675612,  0.3875355 ,  0.42867032,  0.02635409,
        0.66556603,  0.23359237, -0.45242714,  0.67216407, -0.53

In [14]:
'''
    feature「outlook」が
    どのユニットに対して影響が強いかを参照
'''
threshold = 0.65

outlook_index = np.where(outlook > threshold)
outlook_index

(array([ 9, 19, 47, 56, 65, 68]),)

In [15]:
'''
    同様に「メール」「編集」「可能」「未読」「探す」「大変」についても
    影響の強いユニットを参照
'''
mail   = estimator.coefs_[0][43]  # ('メール', 43)
henshu = estimator.coefs_[0][105] # ('未読', 105)
kanou  = estimator.coefs_[0][78]  # ('可能', 78)
midoku = estimator.coefs_[0][123] # ('編集', 123)
sagasu = estimator.coefs_[0][100] # ('探す', 100)
taihen = estimator.coefs_[0][85]  # ('大変', 85)

mail_index   = np.where(mail   > threshold)
henshu_index = np.where(henshu > threshold)
kanou_index  = np.where(kanou  > threshold)
midoku_index = np.where(midoku > threshold)
sagasu_index = np.where(sagasu > threshold)
taihen_index = np.where(taihen > threshold)

In [16]:
mail_index

(array([13, 87]),)

In [17]:
henshu_index

(array([43, 49, 57, 61, 80, 81]),)

In [18]:
kanou_index

(array([65, 99]),)

In [19]:
midoku_index

(array([ 8, 17, 20, 40, 49, 64, 65, 68, 87]),)

In [20]:
sagasu_index

(array([ 5, 43, 54, 61, 77]),)

In [21]:
taihen_index

(array([34, 43, 57, 77]),)

### (3-3) MLP内におけるユニット〜ラベル間の重み付けを確認

MLPにおいては、coefs\_ という変数に、学習結果（最適化された影響変数）が格納されています。

coefs\_ の出力レイヤーに対応する要素（estimator.coefs\_[1]）は、ユニット数（100件）× ラベル数（28件）の２次元配列になっています。

In [22]:
np.size(estimator.coefs_[1], axis=0) # <--- ユニットの数になります

100

In [23]:
np.size(estimator.coefs_[1], axis=1) # <--- ラベルの数になります

28

こちらの estimator.coefs\_[1] を参照すると、各々のユニットが、ラベルに対して、どれだけ影響の強さを持っているか（＝重み付けがされているか）が判断できるかと存じます。

逆の見方をすると・・・各々のラベルが、どのユニットから誘導されやすいか、ということを判断する手助けになるかと考えております。

In [24]:
'''
    出力レイヤーにおける
    ラベル「3402」のインデックスを取得
'''
np.where(estimator.classes_==3402)

(array([6]),)

In [25]:
'''
     出力レイヤーのユニットごとの影響変数を
     ラベル 3402 について確認
     feature=[2010 outlook メール 編集 可能]
'''
seikai = estimator.coefs_[1].T[6] # ラベル「3402」のインデックス
seikai

array([-0.73427055,  0.09419913, -0.48899394,  0.53353946, -0.60901666,
        0.52237497,  0.53409747, -0.50028231,  0.45903818,  0.12720822,
       -0.48754021,  0.66948241, -0.61553091,  0.67418052,  0.45538165,
        0.58663356,  0.38503268,  0.57948044, -0.72842948, -0.52500183,
        0.68204944, -0.47774884,  0.4969988 , -0.47273315,  0.60967857,
       -0.51283855,  0.50472494, -0.52493643,  0.58204477, -0.58784803,
       -0.7174271 ,  0.49477265,  0.20554232,  0.67314168,  0.58046522,
       -0.59155928, -0.75557716, -0.46257726,  0.63764514, -0.39116489,
        0.61652158, -0.6105198 ,  0.65160078, -0.43197592, -0.45509462,
       -0.59426682, -0.53711528,  0.61460633,  0.41296938,  0.65536106,
       -0.38413698,  0.54110396,  0.50084029,  0.41541234, -0.61694399,
       -0.60481112, -0.59626975,  0.50805439,  0.35771922, -0.5263271 ,
       -0.61406118, -0.50527493,  0.53414702,  0.54710729,  0.51628839,
        0.69104782,  0.63289734, -0.50585279,  0.59684082, -0.47

In [26]:
'''
    feature「outlook」「メール」「編集」「可能」「未読」「探す」「大変」の影響が強いユニットについて、
    出力レイヤーの影響変数を抽出してみます
'''
def get_output_layer_coef(coefs, indexes, threshold):
    __coef = coefs[indexes]
    return __coef[__coef > threshold] # thresholdを超えているものだけ抽出

threshold = 0.6 # <---全般的に影響変数の値が大きいので調整しております

In [27]:
get_output_layer_coef(seikai, outlook_index, threshold) # <---ラベルに関係のあるfeature

array([ 0.61460633,  0.69104782])

In [28]:
get_output_layer_coef(seikai, mail_index, threshold) # <---ラベルに関係のあるfeature

array([ 0.67418052])

In [29]:
get_output_layer_coef(seikai, henshu_index, threshold) # <---ラベルに関係のあるfeature

array([ 0.65536106])

In [30]:
get_output_layer_coef(seikai, kanou_index, threshold) # <---ラベルに関係のあるfeature

array([ 0.69104782,  0.6015781 ])

ラベルに無関係な feature に、なぜか重み付けがされているのですが、サンプル数が少ないせいかもしれません。

ただし、下記featureが質問文に使われない限り、回答には影響がないかと存じます。

In [31]:
get_output_layer_coef(seikai, midoku_index, threshold) # <---ラベルに無関係

array([ 0.68204944,  0.61652158,  0.65536106,  0.69104782])

In [32]:
get_output_layer_coef(seikai, sagasu_index, threshold) # <---ラベルに無関係

array([], dtype=float64)

In [33]:
get_output_layer_coef(seikai, taihen_index, threshold) # <---ラベルに無関係

array([], dtype=float64)

#### 類似質問文のラベル 3940 についても見てみます。

In [34]:
'''
    出力レイヤーにおける
    ラベル「3940」のインデックスを取得
'''
np.where(estimator.classes_==3940)

(array([26]),)

In [35]:
'''
     出力レイヤーのユニットごとの影響変数を
     ラベル 3940 について確認
     feature=[2010 outlook メール 未読 探す 大変]
'''
ruiji = estimator.coefs_[1].T[26] # ラベル「3940」のインデックス
ruiji

array([-0.52082929, -0.54828001,  0.43470946, -0.52561335, -0.51215141,
        0.40249564,  0.39776507, -0.41412299,  0.40721943,  0.44357679,
       -0.37425573, -0.47017284, -0.45784436,  0.43831028,  0.47370524,
       -0.23469131,  0.43282635,  0.05848842, -0.47908162, -0.47403543,
        0.46095744, -0.47386384, -0.45734288, -0.51008361, -0.48314942,
       -0.44441993,  0.40069439, -0.05635786, -0.42560723, -0.49959014,
        0.43002493, -0.48495894,  0.47883849,  0.50257127,  0.4063876 ,
        0.39950284,  0.44588818, -0.4888365 , -0.50011189, -0.20997357,
        0.43572999, -0.52254239, -0.4773166 ,  0.41780976, -0.52963929,
        0.48004788, -0.50297498, -0.49173775,  0.44601758,  0.36363201,
        0.05681817,  0.48396543, -0.51424188,  0.50921994,  0.46975785,
       -0.45033825,  0.44998926,  0.34029927, -0.41643834, -0.51138248,
       -0.45653783,  0.41943443,  0.48347621,  0.42168155,  0.47407428,
       -0.50794261, -0.45067554, -0.49103275, -0.51004052, -0.52

In [36]:
'''
    feature「outlook」「メール」「編集」「可能」「未読」「探す」「大変」の影響が強いユニットについて、
    出力レイヤーの影響変数を抽出してみます
'''
threshold = 0.43 # <---全般的に影響変数の値が小さいので調整しております

In [37]:
get_output_layer_coef(ruiji, outlook_index, threshold) # <---ラベルに関係のあるfeature

array([ 0.44357679,  0.44998926])

In [38]:
get_output_layer_coef(ruiji, mail_index, threshold) # <---ラベルに関係のあるfeature

array([ 0.43831028])

In [39]:
get_output_layer_coef(ruiji, henshu_index, threshold) # <---ラベルに無関係

array([], dtype=float64)

In [40]:
get_output_layer_coef(ruiji, kanou_index, threshold) # <---ラベルに無関係

array([], dtype=float64)

In [41]:
get_output_layer_coef(ruiji, midoku_index, threshold) # <---ラベルに関係のあるfeature

array([ 0.46095744,  0.43572999,  0.47407428])

In [42]:
get_output_layer_coef(ruiji, sagasu_index, threshold) # <---ラベルに関係のあるfeature

array([ 0.46975785])

In [43]:
get_output_layer_coef(ruiji, taihen_index, threshold) # <---ラベルに関係のあるfeature

array([], dtype=float64)